# Library

In [ ]:
# Numpy, pandas, time
import numpy as np
import pandas as pd
import time

# Iterative Stratification untuk cross validation multilabel
from skmultilearn.model_selection import IterativeStratification

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

#Import Tensorflow dan extension
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.optimizers import Adam, Nadam, Adagrad, SGD, RMSprop, Adadelta

#Import keras tuner dan metrics untuk tuning parameter
import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization, Sklearn
from sklearn import metrics
import tensorflow_addons as tfa

# Function

In [ ]:
#Modifikasi IterativeStratification agar hasil random data tetap sama

def new_init(self, n_splits=3, order=1, sample_distribution_per_fold = None, random_state=None):

                  self.order = order
                  if random_state is not None:
                      do_shuffle = True
                  else:
                      do_shuffle = False
                  super(
                      IterativeStratification,
                      self).__init__(n_splits,
                                     shuffle=do_shuffle,
                                     random_state=random_state)
                  if sample_distribution_per_fold:
                      self.percentage_per_fold = sample_distribution_per_fold
                  else:
                      self.percentage_per_fold = [1 / float(self.n_splits) for _ in range(self.n_splits)]
    
IterativeStratification.__init__ = new_init
# cv = IterativeStratification(n_splits=5, random_state = 123)

In [ ]:
#Fungsi model SAE
def sae_model(xt, xv= None, EPOCHS= 100,BATCH_SIZE= 32, opt= "adam",
              hl_node= 1024, lr= 0.01, af= "relu", num_layers= 3, do= 0.5, fr_node= 0.5,
              verbose = 0,return_fe = False):
  #Setting result placeholders
  xt_ae = [] ;xv_ae = [] ; w_ae = []
  #If validation set is not present, use train set as validation set
  if xv is None :
    xv = xt.copy()
  opt = tf.keras.optimizers.get(opt) #Set optimizer
  K.set_value(opt.learning_rate, lr) #Set learning rate

  #Stacked Autoencoder architecture
  for n_layers in range(num_layers):
    #Autoencoder
    inp = Input(shape=(xt.shape[1],))
    #Apply Dropout
    hidden_layer = Dropout(do)(inp)
    #Layer encoder (jumlah layer sesuai dengan n_layers)
    enc = Dense(int(hl_node*(fr_node**n_layers)), activation = af)(hidden_layer)  
    #Layer Decoder
    dec = Dense(xt.shape[1],activation="linear")(enc)
    ae = Model(inp, dec)
    #Compile model
    ae.compile(optimizer=opt, loss='mean_squared_error')
    #EarlyStop jika sudah konvergen 
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=verbose)
    #Latih model
    ae.fit(xt, xt, 
           epochs=EPOCHS,batch_size=BATCH_SIZE, 
           shuffle=True, callbacks = [es] , verbose = verbose,
           validation_data = (xv,xv))
    #Ekstrak Feature extraction
    fe = Model(ae.input, enc)
    #Simpan data hasil latih
    xt = fe.predict(xt) ; xt_ae.append(xt)
    xv = fe.predict(xv) ; xv_ae.append(xv)
    #Simpan bobot hasil latih SAE
    w_ae.append([layer_name for layer_name in ae.layers if "dense" in layer_name.name][0].get_weights())
    if verbose:
      print("Layer {} trained".format(n_layers+1))

  return (w_ae,xv) if return_fe else w_ae

In [ ]:
#Fungsi DNN

# kelas output 7
# n_outputs = 7

def dnn_model(xt, n_outputs= 7, sae_weights= None, EPOCHS= 100,BATCH_SIZE= 32, opt= "adam",
              hl_node= 1024, lr= 0.01, af= "relu", num_layers= 3, do= 0.5, fr_node= 0.5):
  opt = tf.keras.optimizers.get(opt) #Set optimizer
  K.set_value(opt.learning_rate, lr) #Set learning rate
  
  #Model architecture
  input_layer = Input(shape=(xt.shape[1],))
  hidden_layer = BatchNormalization()(input_layer)
  hidden_layer = Dropout(do)(hidden_layer)

#Set jumlah hidden layer
  for n_layers in range(num_layers):
    hidden_layer = Dense(int(hl_node*(fr_node**n_layers)), activation = af)(hidden_layer)
    hidden_layer = BatchNormalization()(hidden_layer)
    hidden_layer = Dropout(do)(hidden_layer)
  output_layer = Dense(n_outputs, activation = 'sigmoid')(hidden_layer)

#latih model
  dnn = Model(input_layer, output_layer)

  #Latih model DNN dengan bobot SAE (jika bobot ada)
  if sae_weights is not None:
    weights = sae_weights
    dnn_dense = [layer_name for layer_name in dnn.layers if "dense" in layer_name.name]
    for weight_from,weight_to in list(zip(weights,dnn_dense)):
      weight_to.set_weights(weight_from)

#Compile model
  dnn.compile(optimizer=opt, loss='binary_crossentropy', metrics = [tf.keras.metrics.BinaryAccuracy(),
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall()],
               )
  return dnn

In [ ]:
#Fungsi model untuk tuning
def build_model(hp):
    #Isi parameter yang akan dituning
    params = {
              'hl_node' : hp.Choice('units',values= para_hl_node),
              'af' : hp.Choice('activation',values= para_af),
              'lr' : hp.Choice('learning_rate',values= para_lr),
              'opt' : hp.Choice('optimizer',values= para_opt),
              'num_layers' : hp.Choice('num_layers',values= para_num_layers),
              'do' : hp.Choice('dropout_rate',values= para_do),
              'fr_node' : hp.Choice('fraction_node',values= para_fr_node)
              }
    #Latih model SAE
    sae_weights = sae_model(xt = X, xv = X_train, EPOCHS= 100,**params)
    #Latih model DNN dengan bobot SAE
    sae_dnn = dnn_model(X_train, sae_weights=sae_weights, EPOCHS= 100,**params)
    return sae_dnn

# SAE-DNN & DNN FUNCTION


In [ ]:
def res_sae_dnn(X, hl_node, lr, opt, num_layers, do, fr_node):
  
  acc_results_tuned = list()
  f1_results_tuned = list()
  prec_results_tuned = list()
  rec_results_tuned = list()
  n_inputs, n_outputs = X.shape[1], 7
  
  print("finding sae weights....")
  ti0 = time.time()

  sae_weigths_tuned = sae_model(xt = X, hl_node = hl_node, af = "relu", lr = lr, opt= opt, num_layers = num_layers,
                                do = do, fr_node= fr_node)

  ti1 = time.time()
  print('done, processing time:', ti1-ti0)
  
  i=0
  t0 = time.time()
  # enumerate folds
  np.random.seed(123)

  for train_ix, test_ix in cv.split(X,Y):
    X_train, X_test = X.iloc[train_ix,:], X.iloc[test_ix,:]
    y_train, y_test = Y[train_ix], Y[test_ix]
    # define model
    model_tuned = dnn_model(xt = X_train, sae_weights = sae_weigths_tuned, hl_node = hl_node, af = "relu", lr = lr, opt= opt, num_layers = num_layers,
                              do = do, fr_node= fr_node)
    # fit model
    model_tuned.fit(X_train, y_train, verbose=False, epochs=100)
    # make a prediction on the test set
    yhat = model_tuned.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate metrics
    acc = accuracy_score(y_test, yhat)
    f1 = f1_score(y_test, yhat, average='samples')
    prec = precision_score(y_test, yhat, average='samples')
    rec = recall_score(y_test, yhat, average='samples')
    
    # store result
    print("CV number: ", i)
    print('accuracy of :>%.3f' % acc)
    print('F1 of :>%.3f' % f1)
    print('Precision of :>%.3f' % prec)
    print('Recall of :>%.3f' % rec)
    acc_results_tuned.append(acc)
    f1_results_tuned.append(f1)
    prec_results_tuned.append(prec)
    rec_results_tuned.append(rec)
    i=i+1
  
  t1 = time.time()
  total_waktu = t1-t0

  print("waktu proses: ", total_waktu)
  print("Accuracy array:", acc_results_tuned)
  print("F1 array:", f1_results_tuned)
  print("Precision array:", prec_results_tuned)
  print("Recall array:", rec_results_tuned)

  return [acc_results_tuned, f1_results_tuned, prec_results_tuned, rec_results_tuned, model_tuned, total_waktu]

In [ ]:
def dnn_saja(X, hl_node, lr, opt, num_layers, do, fr_node):
  acc_results2 = list()
  f1_results2 = list()
  prec_results2 = list()
  rec_results2 = list()
  n_inputs, n_outputs = X.shape[1], 7
  # define evaluation procedure
  # cv = IterativeStratification(n_splits=5, random_state = 123)
  i=0
  # enumerate folds
  t0 = time.time()

  for train_ix, test_ix in cv.split(X,Y):
    X_train, X_test = X.iloc[train_ix,:], X.iloc[test_ix,:]
    y_train, y_test = Y[train_ix], Y[test_ix]
    # define model tanpa bobot SAE
    model = dnn_model(xt= X_train, sae_weights= None, hl_node= hl_node, af= "relu", lr= lr, opt= opt, num_layers= num_layers,
                              do= do, fr_node= fr_node)
    # fit model
    model.fit(X_train, y_train, verbose=False, epochs=100)
    # make a prediction on the test set
    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate metrics
    acc = accuracy_score(y_test, yhat)
    f1 = f1_score(y_test, yhat, average='samples')
    prec = precision_score(y_test, yhat, average='samples')
    rec = recall_score(y_test, yhat, average='samples')
    # store result
    print("CV number: ", i)
    print('accuracy of :>%.3f' % acc)
    print('F1 of :>%.3f' % f1)
    print('Precision of :>%.3f' % prec)
    print('Recall of :>%.3f' % rec)
    acc_results2.append(acc)
    f1_results2.append(f1)
    prec_results2.append(prec)
    rec_results2.append(rec)
    i=i+1
  
  t1 = time.time()
  total_waktu = t1-t0
  print("waktu proses", total_waktu)
  print("Accuracy array:", acc_results2)
  print("F1 array:", f1_results2)
  print("Precision array:", prec_results2)
  print("Recall array:", rec_results2)

  return [acc_results2, f1_results2, prec_results2, rec_results2, model, total_waktu]

        

# MAIN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load dataset
df_pubchem = pd.read_csv('/content/drive/MyDrive/skripsi/df_pubchem_rapi.csv')
df_maccs = pd.read_csv('/content/drive/MyDrive/skripsi/df_maccs_rapi.csv')
df_klekota = pd.read_csv('/content/drive/MyDrive/skripsi/df_klekota_rapi.csv')

#drop CID_senyawa
X_pubchem = df_pubchem.drop(['CID_senyawa'], axis=1)
X_maccs = df_maccs.drop(['CID_senyawa'], axis=1)
X_klekota = df_klekota.drop(['CID_senyawa'], axis=1)

Y = pd.read_csv('/content/drive/MyDrive/skripsi/kelas_data.csv')
Y = np.array(Y)

In [ ]:
# define evaluation procedure
np.random.seed(123)
#Inisialisasi CV
cv = IterativeStratification(n_splits=5, random_state = 123)

In [ ]:
X = X_maccs

In [ ]:
para_hl_node = [320, 640, 1280, 1500, 1600, 1700, 1800]
para_af = ["relu"]
para_lr = [x for x in np.linspace(0.01,0.1)]
para_opt = ["adam", "adagrad"]
para_num_layers = [2,3,4,5]
para_do = [0.5,0.6,0.7,0.8]
para_fr_node = [0.5,0.66,0.75]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)
#hypterparameter tuning
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_recall',patience = 25)
# Fungsi Bayesian Optimization di Keras Tuner.
tuner = BayesianOptimization(build_model,
    # Metrik yang dicari optimalnya
    objective= kt.Objective("val_recall", direction="max"), 
    # Jumlah percobaan
    max_trials=20,
    executions_per_trial=2,
    # Folder simpan hasil tuning
    directory='/home/fadilrisdian/skripsi-fadil',
    project_name='sae_dnn_tuning_bayesFiltered40k', overwrite = True)
#Jalankan Keras Tuner u
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test),callbacks=[stop_early])
#Tampilkan hasil terbaik
tuner.results_summary()

Trial 20 Complete [00h 02m 56s]
val_recall: 0.8700819611549377

Best val_recall So Far: 0.9409835934638977
Total elapsed time: 01h 12m 43s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in /home/fadilrisdian/skripsi-fadil/sae_dnn_tuning_bayesFiltered40k
Showing 10 best trials
Trial summary
Hyperparameters:
units: 1280
activation: relu
learning_rate: 0.1
optimizer: adam
num_layers: 2
dropout_rate: 0.5
fraction_node: 0.66
Score: 0.9409835934638977
Trial summary
Hyperparameters:
units: 1280
activation: relu
learning_rate: 0.1
optimizer: adam
num_layers: 2
dropout_rate: 0.5
fraction_node: 0.5
Score: 0.9000000059604645
Trial summary
Hyperparameters:
units: 1280
activation: relu
learning_rate: 0.1
optimizer: adam
num_layers: 2
dropout_rate: 0.5
fraction_node: 0.66
Score: 0.8913934230804443
Trial summary
Hyperparameters:
units: 1280
activation: relu
learning_rate: 0.1
optimizer: adam
num_layers: 2
dropout_rate: 0.5
fraction_node: 0.5
Score: 0.8860655725002289
Trial summary
Hype

In [ ]:
#Parameter
X = X_maccs
hl_node = 1800
fr_node = 0.66
num_layers = 2
opt = "adam"
lr = 0.1
do = 0.5


In [ ]:
hasil_saednn = res_sae_dnn(X, hl_node, lr, opt, num_layers, do, fr_node)
dnn_aja = dnn_saja(X, hl_node, lr, opt, num_layers, do, fr_node)

finding sae weights....
done, processing time: 41.210047245025635


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  0
accuracy of :>0.704
F1 of :>0.707
Precision of :>0.709
Recall of :>0.707


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  1
accuracy of :>0.839
F1 of :>0.860
Precision of :>0.872
Recall of :>0.858


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  2
accuracy of :>0.842
F1 of :>0.857
Precision of :>0.870
Recall of :>0.853
CV number:  3
accuracy of :>0.689
F1 of :>0.847
Precision of :>0.819
Recall of :>0.917
CV number:  4
accuracy of :>0.812
F1 of :>0.883
Precision of :>0.878
Recall of :>0.908
waktu proses:  371.9011871814728
Accuracy array: [0.7037037037037037, 0.8390501319261213, 0.8415929203539823, 0.6892857142857143, 0.812223206377325]
F1 array: [0.7071428571428572, 0.8597646270469489, 0.8571849978929624, 0.8465745464852606, 0.883447214138091]
Precision array: [0.708994708994709, 0.8720316622691293, 0.8699115044247787, 0.8186011904761904, 0.8782108060230293]
Recall array: [0.7070399764844209, 0.8577543242450894, 0.8526401179941003, 0.9173086734693876, 0.9076616474756422]
CV number:  0
accuracy of :>0.828
F1 of :>0.891
Precision of :>0.891
Recall of :>0.913
CV number:  1
accuracy of :>0.771
F1 of :>0.851
Precision of :>0.838
Recall of :>0.890
CV number:  2
accuracy of :>0.795
F1 of :>0.884
Precision of :>0.871
Recal

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

print('===================================')
print('===================================')
print('SAE-DNN TUNED PERFORMANCE')
print('Accuracy    : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[0]), np.std(hasil_saednn[0])))
print('F1 Score    : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[1]), np.std(hasil_saednn[1])))
print('Precision   : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[2]), np.std(hasil_saednn[2])))
print('Recall      : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[3]), np.std(hasil_saednn[3])))
print('===================================')
print('===================================')
print('DNN ONLY PERFORMANCE')
print('Accuracy    : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[0]), np.std(dnn_aja[0])))
print('F1 Score    : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[1]), np.std(dnn_aja[1])))
print('Precision   : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[2]), np.std(dnn_aja[2])))
print('Recall       : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[3]), np.std(dnn_aja[3])))


SAE-DNN TUNED PERFORMANCE
Accuracy    : 0.77717±0.067
F1 Score    : 0.83082±0.063
Precision   : 0.82955±0.064
Recall      : 0.84848±0.075
DNN ONLY PERFORMANCE
Accuracy    : 0.80166±0.028
F1 Score    : 0.85476±0.037
Precision   : 0.85373±0.036
Recall       : 0.87507±0.049


In [ ]:
sae_dnn_maccs_tuned = hasil_saednn[4]
dnn_maccs_tuned = dnn_aja[4]

sae_dnn_maccs_tuned.save("sae_dnn_maccs_tuned.h5")
dnn_maccs_tuned.save("dnn_maccs_tuned.h5")

In [ ]:
SD_ACC = []
SD_F1 = []
SD_PRE = []
SD_RE = []

S_ACC = []
S_F1 = []
S_PRE = []
S_RE = []

In [ ]:
t_awal = time.time()
for i in range(8):
  hasil_saednn = res_sae_dnn(X, hl_node, lr, opt, num_layers, do, fr_node)
  dnn_aja = dnn_saja(X, hl_node, lr, opt, num_layers, do, fr_node)
  
  SD_ACC.append(hasil_saednn[0])
  SD_F1.append(hasil_saednn[1])
  SD_PRE.append(hasil_saednn[2])
  SD_RE.append(hasil_saednn[3])

  S_ACC.append(dnn_aja[0])
  S_F1.append(dnn_aja[1])
  S_PRE.append(dnn_aja[2])
  S_RE.append(dnn_aja[3])
  
t_akhir = time.time()
total_waktu = t_akhir - t_awal
print("total waktu adalah" + str(total_waktu))

finding sae weights....
done, processing time: 88.39249634742737


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  0
accuracy of :>0.434
F1 of :>0.751
Precision of :>0.680
Recall of :>0.901


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  1
accuracy of :>0.674
F1 of :>0.809
Precision of :>0.776
Recall of :>0.910


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  2
accuracy of :>0.751
F1 of :>0.766
Precision of :>0.767
Recall of :>0.776


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  3
accuracy of :>0.775
F1 of :>0.791
Precision of :>0.792
Recall of :>0.804
CV number:  4
accuracy of :>0.777
F1 of :>0.791
Precision of :>0.795
Recall of :>0.796
waktu proses 346.57118797302246
Accuracy array: [0.7542147293700089, 0.8046594982078853, 0.7513134851138353, 0.7745098039215687, 0.7769028871391076]
F1 array: [0.8501499978873538, 0.8531874039938556, 0.7656054540905679, 0.7906692980222392, 0.7914489855434738]
Precision array: [0.8298728186926945, 0.8545101553166068, 0.7667688266199649, 0.7922650878533232, 0.7954359871682706]
Recall array: [0.9168884945282461, 0.8698775388291518, 0.7758464681844717, 0.8039894745777099, 0.7961213181685622]
total waktu adalah6127.071844339371


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

print('===================================')
print('===================================')
print('SAE-DNN TUNED PERFORMANCE')
print('Accuracy    : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[0]), np.std(hasil_saednn[0])))
print('F1 Score    : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[1]), np.std(hasil_saednn[1])))
print('Precision   : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[2]), np.std(hasil_saednn[2])))
print('Recall      : {0:.5f}±{1:.3f}'.format(np.mean(hasil_saednn[3]), np.std(hasil_saednn[3])))
print('===================================')
print('===================================')
print('DNN ONLY PERFORMANCE')
print('Accuracy    : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[0]), np.std(dnn_aja[0])))
print('F1 Score    : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[1]), np.std(dnn_aja[1])))
print('Precision   : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[2]), np.std(dnn_aja[2])))
print('Recall       : {0:.5f}±{1:.3f}'.format(np.mean(dnn_aja[3]), np.std(dnn_aja[3])))


SAE-DNN TUNED PERFORMANCE
Accuracy    : 0.83898±0.019
F1 Score    : 0.87019±0.016
Precision   : 0.87679±0.018
Recall      : 0.87866±0.015
DNN ONLY PERFORMANCE
Accuracy    : 0.81722±0.029
F1 Score    : 0.84732±0.033
Precision   : 0.85317±0.036
Recall       : 0.85633±0.033


In [ ]:
SD_ACC = []
SD_F1 = []
SD_PRE = []
SD_RE = []

S_ACC = []
S_F1 = []
S_PRE = []
S_RE = []

In [ ]:
import csv

In [ ]:
fields = ['Name', 'Branch', 'Year', 'CGPA'] 

In [ ]:
for i in range(8):
  print(i)

0
1
2
3
4
5
6
7


In [ ]:
#percobaan 10 kali
akurasi = []
f1_score = []
precision = []
recall = []

akurasi_std = []
f1_score_std = []
precision_std = []
recall_std = []

akur_std = []
for i in range(10):
  akurasi.append(np.mean(S_ACC[i]))
  f1_score.append(np.mean(S_F1[i]))
  precision.append(np.mean(S_PRE[i]))
  recall.append(np.mean(S_RE[i]))
  
  akurasi_std.append(np.std(S_ACC[i]))
  f1_score_std.append(np.std(S_F1[i]))
  precision_std.append(np.std(S_PRE[i]))
  recall_std.append(np.std(S_RE[i]))

akhir_saednn = [akurasi, f1_score, precision, recall, akurasi_std, f1_score_std, precision_std, recall_std]

In [ ]:
import pandas as pd


In [ ]:
df = pd.DataFrame(akurasi, columns=['Accuracy'])

In [ ]:
df['f1_score'] = f1_score
df['precision'] = precision
df['recall'] = recall
df['akurasi_std'] = akurasi_std
df['f1_score_std'] = f1_score_std
df['precision_std'] = precision_std
df['recall_std'] = recall_std

df

In [ ]:
df.to_csv('Pubchem_1-7_DNN.csv', index=False)

In [ ]:
model_tuned.save("sae_dnn_maccs_final.h5")
model.save("dnn_maccs_final.h5")